In [34]:
%run runaway_functionsv3

In [31]:
cl = Cluster("Alessi_43")
run = cl.runaways(temp_threshold=10000)
run
# ra = cl.runaways_all()
# thiso = cl.theoretical_isochrone()
# ra['Temp. Est'] = ra["Teff"]



0.9 6.8 -0.1


RA_ICRS_1,DE_ICRS_1,rgeo,Teff,Temp. Est,e_RA_ICRS,e_DE_ICRS,_r_1,HIP,TYC2,Source,Plx,e_Plx,pmRA,pmDE,e_pmRA,e_pmDE,RUWE,Gmag,BP-RP,BPmag,RPmag,b_rgeo,B_rgeo,e_Gmag,e_BPmag,e_RPmag,e_BP-RP,SkyCoord,rmRA,rmDE,v_pec,logg,RV,e_RV,FG,e_FG,FBP,e_FBP,FRP,e_FRP,RAVE5,RAVE6
deg,deg,pc,K,K,mas,mas,,,,,mas,mas,mas / yr,mas / yr,mas / yr,mas / yr,,mag,mag,mag,mag,pc,pc,,,,,"deg,deg,pc",mas / yr,mas / yr,km / s,log(cm.s**-2),km / s,km / s,,,,,,,,
float64,float64,float64,float64,float64,float64,float64,float64,int32,str11,int64,float64,float32,float64,float64,float32,float32,float64,float64,float64,float64,float64,float64,float64,float64,float32,float32,float32,SkyCoord,float64,float64,float64,float64,float64,float32,float64,float32,float32,float32,float32,float32,str1,str1
132.26918571419,-41.86012579175,814.32153300,8231.5,10240.0,0.0194,0.0219,17.6322,--,--,5524602036116091520,1.1904,0.0254,-10.707,7.029,0.026,0.029,1.723,11.258210,0.356719,11.384149,11.027430,796.94012500,831.44531200,0.002774,0.002865,0.003795,0.00666,"132.26918571419,-41.86012579175,814.321533",-5.176279289245606,3.10908074760437,23.30687698503049,4.0056,31.10,2.85,591102.51261,175.7,3.817e+05,229,3.077e+05,96.41,--,--


In [44]:
from astropy.wcs import WCS
from astropy.io import fits
from astropy import units as u
from astropy.table import Table
from astropy.time import Time
import matplotlib.pyplot as plt
from astropy.coordinates import SkyCoord, Angle
from regions import CircleSkyRegion, PointSkyRegion, LineSkyRegion
import numpy as np

# Your previous code to calculate coordinates
allrun = cl.runaways()[:]
allrun_coord_now = SkyCoord(ra=allrun['RA_ICRS_1'], 
                        dec=allrun['DE_ICRS_1'],
                        distance=allrun['rgeo'], 
                        pm_ra_cosdec=allrun['rmRA'],
                        pm_dec=allrun['rmDE'],
                        obstime=Time('J2000')+500*u.kyr)

allrun_coord_earlier = allrun_coord_now.apply_space_motion(dt=-100*u.kyr)



def get_search_region(cluster, extra=10,display=True,**kwargs):
    """
    Plots and saves the fits file for the region around the given cluster.

    Parameters:
    - extra (float): Additional extent for the search region (default is from config['Cluster']['search_extent']).

    Returns:
    None
    """
    search_arcmin = cl.search_arcmin
    
    # Define the file path
    fits_file_path = f'./Clusters/{cl.name}/{cl.name}_extra{extra}pc.fits'
    
    # Check if the file already exists
    if os.path.exists(fits_file_path):
        print(f'fits image exists in {cl.name} folder')
        # File exists, no need to download, use the existing file
        images = [fits.open(fits_file_path)]
        # Extract the WCS information
        wcs = WCS(images[0][0].header)
    else:
        # File doesn't exist, get the image data from SkyView
        images = SkyView.get_images(position=cl.skycoord,
                                    survey='DSS',
                                    radius=2*search_arcmin,
                                    **kwargs)
        # Extract the WCS information
        wcs = WCS(images[0][0].header)
        hdu = fits.PrimaryHDU(data=images[0][0].data, header=images[0][0].header)
        hdulist = fits.HDUList([hdu])
        
        # Save the fits file
        hdulist.writeto(fits_file_path, overwrite=True)
# Open the FITS file and extract the image and WCS
cluster_10pc_fits_path = f'./Clusters/{cl.name}/{cl.name}_extra10pc.fits'
if not os.path.exists(cluster_10pc_fits_path):
    get_search_region(cl)
with fits.open(cluster_10pc_fits_path) as fits_file:
    image = fits_file[0]
    wcs = WCS(image.header)
    fig, ax = plt.subplots(subplot_kw={'projection': wcs}, figsize=(16, 16))
    ax.imshow(image.data, cmap='gray')
    ax.set_xlabel('Right Ascension (hms)')
    ax.set_ylabel('Declination (degrees)')

    # Plot the cluster region
    c = cl.skycoord
    radius = cl.r50
    region = CircleSkyRegion(c, radius)
    region_pix = region.to_pixel(wcs)
    region_pix.plot(ax=ax, color='red', lw=2)
    
    # Calculate the pixel coordinates of the runaway stars
    allrun_pixels_now = wcs.world_to_pixel(allrun_coord_now)
    allrun_pixels_earlier = wcs.world_to_pixel(allrun_coord_earlier)

    # Plot the current positions as scatter points
    scatter_main = ax.scatter(allrun_pixels_now[0], allrun_pixels_now[1], c=allrun['Temp. Est'], cmap='spring_r', s=50)
    colorbar = fig.colorbar(scatter_main,ax=ax)       # Plot the lines showing motion
    for start, end in zip(np.transpose(allrun_pixels_now), np.transpose(allrun_pixels_earlier)):
        ax.plot([start[0], end[0]], [start[1], end[1]], color='blue')
plt.tight_layout()
plt.show()


0.9 6.8 -0.1


/usr/lib/python3/dist-packages/erfa/core.py:154: ErfaWarning: ERFA function "taiutc" yielded 1 of "dubious year (Note 4)"
  warnings.warn('ERFA function "{}" yielded {}'.format(func_name, wmsg),


In [37]:
dias2021 = Table.read("dias2021.tsv", format="ascii.ecsv")
maskplx = dias2021['Plx'] > 0.3
maskage = dias2021['logage'] < 7.7
workclusters = []
for clustername in dias2021[maskplx & maskage][:]['Cluster']:
    if clustername not in ['ASCC_79','BH_164','BH_23','Collinder_135','Collinder_140','Gulliver_9','IC_2391','IC_2602','Mamajek_1','Platais_8','UPK_535','UPK_606','UPK_640','Berkeley_59','COIN-Gaia_37','Ivanov_4','LP_1937','Sigma_Ori','UBC_632']:
        workclusters.append(clustername)
        # print(clustername)
        # cl = Cluster(clustername)
        # cl.prepare_trace()
# cl.stars_in_region()
# cl.stars_in_region()

In [ ]:
workclusters_mask = [cluster in workclusters for cluster in dias2021['Cluster']]
workclusterstable = dias2021[workclusters_mask]

rvexists = workclusterstable['RV'].mask
nrvgood = workclusterstable['NRV'] > 4

workclusterstable[~rvexists & nrvgood]

In [ ]:
def runaways_from(clustername):
    cl = Cluster(clustername)
    fs4giesler = cl.fast_stars_in_region()
    outputs = os.listdir(f"/home/surodeep/suro_aiu/traceback/cluster_runaway/{cl.name}/runaways/")
    linenos = []
    for output in outputs:
        #print(output)
        if 'run' in output:
            linenos.append(int(output.split("+")[1].replace(".out","")))
    linenos.sort()
    # print(linenos)
    i=np.array(linenos)-3
    def source_of(lineno, input_table):
        return input_table[lineno-2]['Source']
    return fs4giesler[i]
for cluster in workclusters:
    t = runaways_from(cluster)[runaways_from(cluster)['Teff'].argmax()]['Teff']
    
    if t>10000:
        print(cluster, t,runaways_from(cluster)[runaways_from(cluster)['Teff'].argmax()]['Source'])